In [1]:
import pandas as pd          # Biblioteca para trabalhar com tabelas (DataFrames)
import requests              # Biblioteca para acessar páginas da internet

# Endereço da página da Wikipedia que contém a lista do S&P 100
url = "https://en.wikipedia.org/wiki/S%26P_100"

# Cabeçalho que "imita" um navegador comum
# Isso é necessário porque a Wikipedia bloqueia acessos que parecem ser de robôs
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/120.0.0.0 Safari/537.36"
}

# Faz o pedido para acessar a página da Wikipedia
response = requests.get(url, headers=headers)

# Se a página não puder ser acessada (erro 403, 404, etc),
# o programa para aqui e mostra o erro
response.raise_for_status()

# Lê todas as tabelas existentes no HTML da página
# O resultado é uma lista de tabelas (DataFrames)
tables = pd.read_html(response.text)


C:\Users\abgad\AppData\Local\Temp\ipykernel_1144\2834489174.py:24: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


In [2]:
# Inicializamos a variável como None (vazia)
# Ela vai guardar a tabela correta do S&P 100
sp100_table = None

# Percorremos todas as tabelas encontradas na página
for table in tables:
    # Verificamos se essa tabela tem uma coluna chamada "Symbol"
    # Essa coluna contém os códigos das ações (tickers)
    if "Symbol" in table.columns:
        sp100_table = table
        break  # Paramos o loop assim que encontramos a tabela correta

# Se nenhuma tabela com a coluna "Symbol" foi encontrada,
# levantamos um erro para avisar que algo deu errado
if sp100_table is None:
    raise ValueError("Tabela do S&P 100 não encontrada")

# Extraímos apenas a coluna "Symbol" da tabela
# e transformamos em uma lista Python
tickers = sp100_table["Symbol"].tolist()

# Ajuste necessário para o Yahoo Finance:
# alguns tickers usam ponto (.) na Wikipedia,
# mas no Yahoo Finance usam hífen (-)
# Exemplo: BRK.B → BRK-B
tickers = [t.replace(".", "-") for t in tickers]

# Mostra quantos tickers foram extraídos
print(f"{len(tickers)} tickers extraídos")

# Mostra os 10 primeiros tickers como conferência
print(tickers[:10])


101 tickers extraídos
['AAPL', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AIG', 'AMD', 'AMGN', 'AMT', 'AMZN']


In [3]:
import yfinance as yf   # Biblioteca para baixar dados financeiros
import pandas as pd

# Definimos o período desejado
data_inicio = "2015-01-01"
data_fim = "2025-12-31"

# Faz o download dos dados de todas as ações de uma vez
# interval="1d" significa dados diários
# group_by="ticker" organiza os dados por ação
dados = yf.download(
    tickers=tickers,
    start=data_inicio,
    end=data_fim,
    interval="1d",
    group_by="ticker",
    auto_adjust=False,   # Não ajusta automaticamente (vamos usar Adj Close)
    threads=True         # Baixa várias ações em paralelo (mais rápido)
)

# Exibe uma mensagem de confirmação
print("Download dos dados concluído!")


[*********************100%***********************]  101 of 101 completed

Download dos dados concluído!


In [4]:
# Converte o DataFrame com colunas hierárquicas
# para um formato "longo" (organizado)
dados_long = (
    dados
    .stack(level=0)            # transforma tickers em linhas
    .reset_index()
    .rename(columns={"level_1": "Ticker"})
)

# Reorganiza a ordem das colunas (opcional)
dados_long = dados_long[
    ["Date", "Ticker", "Open", "High", "Low", "Close", "Adj Close", "Volume"]
]

# Visualização rápida
print(dados_long.head())

# Salva os dados no formato CSV
dados_long.to_csv(
    "sp100_ohlcv_2015_2025.csv",
    index=False  # não salva o índice (o Date já é coluna)
)

print("Dados completos salvos em CSV!")



C:\Users\abgad\AppData\Local\Temp\ipykernel_1144\333022757.py:5: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  .stack(level=0)            # transforma tickers em linhas


Price       Date Ticker       Open       High        Low      Close  \
0     2015-01-02   AAPL  27.847500  27.860001  26.837500  27.332500   
1     2015-01-02   ABBV  65.440002  66.400002  65.440002  65.889999   
2     2015-01-02    ABT  45.250000  45.450001  44.639999  44.900002   
3     2015-01-02    ACN  89.669998  90.089996  88.430000  88.839996   
4     2015-01-02   ADBE  72.699997  73.199997  71.889999  72.339996   

Price  Adj Close       Volume  
0      24.237558  212818400.0  
1      42.091427    5086100.0  
2      36.418682    3216600.0  
3      74.252609    2021300.0  
4      72.339996    2349200.0  
Dados completos salvos em CSV!


In [5]:
import os

# Criar uma pasta para não bagunçar o computador
os.makedirs("dados_acoes", exist_ok=True)

# Pegar a lista de todos os tickers que estão no tabelão
lista_tickers = dados_long['Ticker'].unique()

for acao in lista_tickers:
    tabela_individual = dados_long[dados_long['Ticker'] == acao]
    
    # Cria o arquivo com o nome da ação (ex: AAPL.csv)
    tabela_individual.to_csv(f"dados_acoes/{acao}.csv", index=False)

print(f"{len(lista_tickers)} arquivos foram separados na pasta 'dados_acoes'.")

101 arquivos foram separados na pasta 'dados_acoes'.


In [6]:
import shutil
# Cria um arquivo .zip com tudo o que tem na pasta
shutil.make_archive('dados_acoes', 'zip', 'dados_acoes')

'c:\\Users\\abgad\\Grupo-1--FinQuant\\dados_acoes.zip'